In [ ]:
import os
import pickle
import numpy as np
import pandas as pd
from collections import Counter
from sqlalchemy import create_engine

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import one_hot_encoder
from sklearn.impute import KNNImputer
from sklearn.metrics import confusion_matrix

In [ ]:
engine = create_engine('sqlite:///../data/titanic.db')
sqlite_connection = engine.connect()
pd.read_sql('SELECT * FROM sqlite_schema WHERE type="table"', con=sqlite_connection)

In [ ]:
np.random.seed(42)

df = pd.read_sql('SELECT * FROM tbl_passengers', con=sqlite_connection)

targets = pd.read_sql('SELECT * FROM tbl_targets', con=sqlite_connection)

# df, targets = fetch_openml("titanic", version=1, as_frame=True, return_X_y=True)

# parch = Parents/Children, sibsp = Siblings/Spouses
df['family_size'] = df['parch'] + df['sibsp']
df['is_alone'] = [1 if family_size==1 else 0 for family_size in df['family_size']]

df['title'] = [name.split(',')[1].split('.')[0].strip() for name in df['name']]
rare_titles = {k for k,v in Counter(df['title']).items() if v < 10}
df['title'] = ['rare' if title in rare_titles else title for title in df['title']]

df = df[[
    'pclass', 'sex', 'age', 'ticket', 'family_size',
    'fare', 'embarked', 'is_alone', 'title'
]]

targets = [int(v) for v in targets['is_survived']]
X_train, X_test, y_train, y_test = train_test_split(df, targets, stratify=targets, test_size=0.2)

In [ ]:
X_train_categorical = X_train[['embarked', 'sex', 'pclass', 'title', 'is_alone']]
X_test_categorical = X_test[['embarked', 'sex', 'pclass', 'title', 'is_alone']]

one_hot_encoder = one_hot_encoder(handle_unknown='ignore', sparse=False).fit(X_train_categorical)
X_train_categorical_one_hot = one_hot_encoder.transform(X_train_categorical)
X_test_categorical_one_hot = one_hot_encoder.transform(X_test_categorical)

In [ ]:
X_train_numerical = X_train[['age', 'fare', 'family_size']]
X_test_numerical = X_test[['age', 'fare', 'family_size']]
knn_imputer = KNNImputer(n_neighbors=5).fit(X_train_numerical)
X_train_numerical_imputed = knn_imputer.transform(X_train_numerical)
X_test_numerical_imputed = knn_imputer.transform(X_test_numerical)

In [ ]:
robust_scaler = RobustScaler().fit(X_train_numerical_imputed)
X_train_numerical_imputed_scaled = robust_scaler.transform(X_train_numerical_imputed)
X_test_numerical_imputed_scaled = robust_scaler.transform(X_test_numerical_imputed)

In [ ]:
X_train_processed = np.hstack((X_train_categorical_one_hot, X_train_numerical_imputed_scaled))
X_test_processed = np.hstack((X_test_categorical_one_hot, X_test_numerical_imputed_scaled))

In [ ]:
model = LogisticRegression(random_state=0).fit(X_train_processed, y_train)
y_train_estimation = model.predict(X_train_processed)
y_test_estimation = model.predict(X_test_processed)

In [ ]:
cm_train = confusion_matrix(y_train, y_train_estimation)

In [ ]:
cm_test = confusion_matrix(y_test, y_test_estimation)

In [ ]:
cm_train

In [ ]:
cm_test

In [ ]:
def do_test(filename, data):
    if not os.path.isfile(filename):
        pickle.dump(data, open(filename, 'wb'))
    truth = pickle.load(open(filename, 'rb'))
    try:
        np.testing.assert_almost_equal(data, truth)
        print(f'{filename} test passed')
    except AssertionError as ex:
        print(f'{filename} test failed {ex}')
    
do_test('../data/cm_test.pkl', cm_test)
do_test('../data/cm_train.pkl', cm_train)
do_test('../data/X_train_processed.pkl', X_train_processed)
do_test('../data/X_test_processed.pkl', X_test_processed)


In [ ]:
def do_pandas_test(filename, data):
    if not os.path.isfile(filename):
        data.to_pickle(filename)
    truth = pd.read_pickle(filename)
    try:
        pd.testing.assert_frame_equal(data, truth)
        print(f'{filename} pandas test passed')
    except AssertionError as ex:
        print(f'{filename} pandas test failed {ex}')
        
# df['title'] = ['asd' for v in df['title']]
do_pandas_test('../data/df.pkl', df)

In [ ]:
rare_titles